<a href="https://colab.research.google.com/github/pupu5003/Vietnamese_Corpus_OCR/blob/main/ocr.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import cv2
import os
import matplotlib.pyplot as plt
import easyocr
import shutil
import json
import re
import zipfile
import numpy as np
from google.colab import files


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 26.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 422.9/422.9 kB 28.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 912.2/912.2 kB 44.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 286.8/286.8 kB 20.5 MB/s eta 0:00:00


In [6]:
from google.colab import files

# Upload the ZIP file
uploaded = files.upload()

# Extract the ZIP file
zip_file = "processed_images.zip"  # Replace with your ZIP file name
extract_to = "/content/processed_images"
os.makedirs(extract_to, exist_ok=True)

with zipfile.ZipFile(zip_file, 'r') as zip_ref:
    zip_ref.extractall(extract_to)

print(f"Files extracted to: {extract_to}")


Saving processed_images.zip to processed_images.zip
Files extracted to: /content/processed_images


In [2]:
# Path to the uploaded ZIP file
zip_file_path = "image.zip"

# Extract to a specific folder
extract_to_folder = "/content"
os.makedirs(extract_to_folder, exist_ok=True)

# Extract the ZIP file
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_to_folder)

print(f"Files extracted to: {extract_to_folder}")


FileNotFoundError: [Errno 2] No such file or directory: 'image.zip'

# Delete Sinonom Black and Red

In [ ]:
# Input and output folders
input_folder = "/content/image/Black"
output_folder = "/content/processed_images"

# Create output folder if it doesn't exist
os.makedirs(output_folder, exist_ok=True)

# Color ranges for black and gray
ranges = [
    (np.array([0, 0, 0]), np.array([180, 255, 30])),
    (np.array([0, 0, 30]), np.array([180, 255, 60])),
    (np.array([0, 0, 60]), np.array([180, 255, 90])),
    (np.array([0, 0, 90]), np.array([180, 255, 120])),
    (np.array([0, 0, 120]), np.array([180, 255, 150]))
]

# Process images
for root, dirs, files in os.walk(input_folder):
    for filename in files:
        input_path = os.path.join(root, filename)
        if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tiff')):
            image = cv2.imread(input_path)
            if image is not None:
                hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
                combined_mask = np.zeros(hsv.shape[:2], dtype=np.uint8)
                for (lower, upper) in ranges:
                    mask = cv2.inRange(hsv, lower, upper)
                    combined_mask = cv2.bitwise_or(combined_mask, mask)
                kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))
                combined_mask = cv2.morphologyEx(combined_mask, cv2.MORPH_CLOSE, kernel)
                combined_mask = cv2.dilate(combined_mask, kernel, iterations=1)
                image_no_black = cv2.inpaint(image, combined_mask, inpaintRadius=3, flags=cv2.INPAINT_TELEA)
                output_subfolder = os.path.join(output_folder, os.path.relpath(root, input_folder))
                os.makedirs(output_subfolder, exist_ok=True)
                output_path = os.path.join(output_subfolder, filename)
                cv2.imwrite(output_path, image_no_black)
                print(f"Processed: {filename}. Saved at: {output_path}")
            else:
                print(f"Cannot read image: {filename}")
        else:
            print(f"Skipped non-image file: {filename}")

print("All images processed successfully.")


Processed: 10729371_page_008.jpeg. Saved at: /content/processed_images/10729371/10729371_page_008.jpeg
Processed: 10729371_page_100.jpeg. Saved at: /content/processed_images/10729371/10729371_page_100.jpeg
Processed: 10729371_page_097.jpeg. Saved at: /content/processed_images/10729371/10729371_page_097.jpeg
Processed: 10729371_page_077.jpeg. Saved at: /content/processed_images/10729371/10729371_page_077.jpeg
Processed: 10729371_page_023.jpeg. Saved at: /content/processed_images/10729371/10729371_page_023.jpeg
Processed: 10729371_page_075.jpeg. Saved at: /content/processed_images/10729371/10729371_page_075.jpeg
Processed: 10729371_page_009.jpeg. Saved at: /content/processed_images/10729371/10729371_page_009.jpeg
Processed: 10729371_page_004.jpeg. Saved at: /content/processed_images/10729371/10729371_page_004.jpeg
Processed: 10729371_page_049.jpeg. Saved at: /content/processed_images/10729371/10729371_page_049.jpeg
Processed: 10729371_page_007.jpeg. Saved at: /content/processed_images/10

In [ ]:
# Input and output folders
input_folder = "/content/image/Red"
output_folder = "/content/processed_images"

# Create output folder if it doesn't exist
os.makedirs(output_folder, exist_ok=True)

# Define color ranges for red and pink in HSV
lower_red1 = np.array([0, 50, 50])
upper_red1 = np.array([10, 255, 255])
lower_red2 = np.array([170, 50, 50])
upper_red2 = np.array([180, 255, 255])
lower_pink = np.array([140, 50, 50])
upper_pink = np.array([170, 255, 255])

# Define kernels for morphological operations
large_kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (15, 15))
small_kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (7, 7))

# Process images in the input folder
for root, dirs, files in os.walk(input_folder):
    for filename in files:
        input_path = os.path.join(root, filename)
        if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tiff')):
            image = cv2.imread(input_path)
            if image is not None:
                # Convert to HSV color space
                hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

                # Create masks for red and pink regions
                mask_red1 = cv2.inRange(hsv, lower_red1, upper_red1)
                mask_red2 = cv2.inRange(hsv, lower_red2, upper_red2)
                mask_pink = cv2.inRange(hsv, lower_pink, upper_pink)

                # Combine red and pink masks
                red_pink_mask = cv2.bitwise_or(mask_red1, mask_red2)
                red_pink_mask = cv2.bitwise_or(red_pink_mask, mask_pink)

                # Apply morphological operations
                dilated_mask = cv2.dilate(red_pink_mask, large_kernel, iterations=1)
                morphed_mask = cv2.morphologyEx(dilated_mask, cv2.MORPH_CLOSE, small_kernel)

                # Apply inpainting to remove red/pink regions
                image_no_red_pink = cv2.inpaint(image, morphed_mask, inpaintRadius=3, flags=cv2.INPAINT_TELEA)

                # Save the processed image
                relative_folder = os.path.relpath(root, input_folder)
                output_subfolder = os.path.join(output_folder, relative_folder)
                os.makedirs(output_subfolder, exist_ok=True)
                output_path = os.path.join(output_subfolder, filename)
                cv2.imwrite(output_path, image_no_red_pink)

                print(f"Processed: {filename}. Saved at: {output_path}")
            else:
                print(f"Cannot read image: {filename}")
        else:
            print(f"Skipped non-image file: {filename}")

print("Finished processing all images.")


Processed: 10695896_page_019.jpeg. Saved at: /content/processed_images/10695896/10695896_page_019.jpeg
Processed: 10695896_page_009.jpeg. Saved at: /content/processed_images/10695896/10695896_page_009.jpeg
Processed: 10695896_page_001.jpeg. Saved at: /content/processed_images/10695896/10695896_page_001.jpeg
Processed: 10695896_page_020.jpeg. Saved at: /content/processed_images/10695896/10695896_page_020.jpeg
Processed: 10695896_page_002.jpeg. Saved at: /content/processed_images/10695896/10695896_page_002.jpeg
Processed: 10695896_page_026.jpeg. Saved at: /content/processed_images/10695896/10695896_page_026.jpeg
Processed: 10695896_page_014.jpeg. Saved at: /content/processed_images/10695896/10695896_page_014.jpeg
Processed: 10695896_page_024.jpeg. Saved at: /content/processed_images/10695896/10695896_page_024.jpeg
Processed: 10695896_page_003.jpeg. Saved at: /content/processed_images/10695896/10695896_page_003.jpeg
Processed: 10695896_page_021.jpeg. Saved at: /content/processed_images/10

# Detect sentence

In [ ]:
import cv2
import os
import easyocr

# Configuration dictionary for different IDs
data_dict = {
    10693040: (12, 400, 250, 10),
    10693454: (11, 400, 200, 10),
    10695896: (12, 200, 290, 10),
    10696073: (10, 400),
    10709453: (15, 7)
}

# Input and output paths
input_folder = '/content/processed_images/10695896'
pdf_id = os.path.basename(input_folder)
output_folder = f'/content/image_crop/{pdf_id}'

# Retrieve parameters for the given ID
sentence_per_image = data_dict[int(pdf_id)][0]
crop_image_wide = data_dict[int(pdf_id)][1]
margin_x = data_dict[int(pdf_id)][2]
width_ths_pass = data_dict[int(pdf_id)][3]

# Create output folder if it doesn't exist
os.makedirs(output_folder, exist_ok=True)

# Function to remove unwanted bounding boxes
def delete_box(boxes):
    i = 0
    while i < len(boxes):
        if abs(boxes[i][0][0][1] - boxes[i][0][2][1]) >= 150:  # Height threshold
            boxes.pop(i)
        elif abs(boxes[i][0][0][0] - boxes[i][0][1][0]) <= 400:  # Width threshold
            boxes.pop(i)
        else:
            i += 1
    return boxes

# Check if one bounding box is within another
def is_box_within(merged_box, non_merged_box):
    return (merged_box[0][0] < non_merged_box[0][0] and
            merged_box[0][1] < non_merged_box[0][1] and
            merged_box[2][0] > non_merged_box[2][0] and
            merged_box[2][1] > non_merged_box[2][1])

# Initialize OCR reader
reader = easyocr.Reader(['vi', 'en'])

# Process each image in the input folder
for filename in os.listdir(input_folder):
    if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
        image_path = os.path.join(input_folder, filename)
        image = cv2.imread(image_path)

        # Perform OCR
        results = reader.readtext(image_path, height_ths=3, slope_ths=6, width_ths=width_ths_pass)

        # Extract bounding boxes and sort by vertical position
        boxes = [(result[0], result[1]) for result in results]
        boxes.sort(key=lambda x: x[0][0][1])

        # Merge bounding boxes based on proximity
        merged_boxes = []
        current_group = []
        variance_threshold = 40  # Y-coordinate proximity threshold

        for bbox, text in boxes:
            if not current_group:
                current_group.append((bbox, text))
            else:
                if abs(bbox[0][1] - current_group[0][0][0][1]) <= variance_threshold:
                    current_group.append((bbox, text))
                else:
                    if len(current_group) > 1:
                        x_min = min([point[0] for box, _ in current_group for point in box])
                        y_min = min([point[1] for box, _ in current_group for point in box])
                        x_max = max([point[0] for box, _ in current_group for point in box])
                        y_max = max([point[1] for box, _ in current_group for point in box])
                        merged_boxes.append([[x_min, y_min], [x_max, y_min], [x_max, y_max], [x_min, y_max]])
                    else:
                        for box, _ in current_group:
                            merged_boxes.append(box)
                    current_group = [(bbox, text)]

        if len(current_group) > 1:
            x_min = min([point[0] for box, _ in current_group for point in box])
            y_min = min([point[1] for box, _ in current_group for point in box])
            x_max = max([point[0] for box, _ in current_group for point in box])
            y_max = max([point[1] for box, _ in current_group for point in box])
            merged_boxes.append([[x_min, y_min], [x_max, y_min], [x_max, y_max], [x_min, y_max]])
        else:
            for box, _ in current_group:
                merged_boxes.append(box)

        res = []
        # Remove boxes that are within other boxes
        for i in range(len(merged_boxes)):
            ok = False
            for j in range(len(merged_boxes)):
                if i != j and is_box_within(merged_boxes[j], merged_boxes[i]):
                    ok = True
                    break
            if not ok:
                res.append(merged_boxes[i])

        merged_boxes = res

        # all_boxes = merged_boxes + non_merged_boxes
        merged_boxes.sort(key=lambda x: (x[0][1], x[0][0]))
        # Create a subfolder for the current file
        filename_without_extension = os.path.splitext(filename)[0]
        file_output_folder = os.path.join(output_folder, filename_without_extension)
        if not os.path.exists(file_output_folder):
            os.makedirs(file_output_folder)

        index = 0
        # Crop and save each box as an image
        for box in merged_boxes:
            # Get the top-left and bottom-right points of the box
            top_left = tuple(map(int, box[0]))
            bottom_right = tuple(map(int, box[2]))
            if bottom_right[1] <= 200:
                continue


            cropped_image = image[top_left[1]-10:bottom_right[1]+10, max(margin_x, top_left[0])-20:bottom_right[0]+20]

            if cropped_image is None or cropped_image.size == 0 or abs(cropped_image.shape[1]) <= crop_image_wide: # x<=700
                # print("Empty image, skipping...")
                continue
            else:
                index = index + 1
                # create folder for each sentence
                cropped_image_folder = os.path.join(file_output_folder, f'cropped_{index:03}')
                if not os.path.exists(cropped_image_folder):
                    os.makedirs(cropped_image_folder)
                cropped_image_path = os.path.join(cropped_image_folder, f'cropped_setence_{index:03}.png')
                cv2.imwrite(cropped_image_path, cropped_image)
                # create folder to save each word of a sentence
                new_file = os.path.join(cropped_image_folder, f'cropped_word')
                if not os.path.exists(new_file):
                    os.makedirs(new_file)
                results_word = reader.readtext(cropped_image_path, height_ths =1.5, slope_ths = 9, width_ths = 0.07)
                word = [(result[0], result[1]) for result in results_word]  # List of (bounding box, text)
                word.sort(key=lambda x: x[0][0][0])
                cnt = 0
                # detect and crop word
                for (bbox, text) in word:
                    cnt = cnt + 1
                    (top_left, top_right, bottom_right, bottom_left) = bbox
                    top_left = tuple(map(int, top_left))
                    bottom_right = tuple(map(int, bottom_right))
                    cropped_word_image = cropped_image[max(0, top_left[1] - 10):min(image.shape[0], bottom_right[1] + 7), top_left[0]:bottom_right[0]]
                    cropped_word_image_path = os.path.join(new_file, f'cropped_word_{cnt:03}.png')
                    cv2.imwrite(cropped_word_image_path, cropped_word_image)
                parts = filename.split('_')
                if len(parts) > 2:  # Ensure there is a "page_" part
                    page_number = parts[2].split('.')[0]
                sentence_index = (int(page_number)-1) * sentence_per_image + index
                if sentence_index % 4 == 1:
                    if cnt != 7:
                        print(f'Error: {filename} - {index} - {cnt} - {sentence_index}')
                elif sentence_index % 4 == 2:
                    if cnt != 7:
                        print(f'Error: {filename} - {index} - {cnt} - {sentence_index}')
                elif sentence_index % 4 == 3:
                    if cnt != 6:
                        print(f'Error: {filename} - {index} - {cnt} - {sentence_index}')
                elif sentence_index % 4 == 0:
                    if cnt != 8:
                        print(f'Error: {filename} - {index} - {cnt} - {sentence_index}')
                # print(f"Image saved to {cropped_image_path}")
        if (index != sentence_per_image): print(file_output_folder)
print("All images have been processed and cropped images have been saved successfully.")


Error: 10695896_page_019.jpeg - 1 - 6 - 217
Error: 10695896_page_019.jpeg - 5 - 9 - 221
Error: 10695896_page_019.jpeg - 12 - 7 - 228
Error: 10695896_page_009.jpeg - 2 - 6 - 98
Error: 10695896_page_009.jpeg - 4 - 7 - 100
Error: 10695896_page_009.jpeg - 5 - 8 - 101
Error: 10695896_page_009.jpeg - 8 - 7 - 104
Error: 10695896_page_001.jpeg - 2 - 6 - 2
Error: 10695896_page_001.jpeg - 5 - 5 - 5
Error: 10695896_page_001.jpeg - 7 - 7 - 7
Error: 10695896_page_001.jpeg - 8 - 10 - 8
Error: 10695896_page_001.jpeg - 10 - 8 - 10
Error: 10695896_page_020.jpeg - 7 - 7 - 235
Error: 10695896_page_002.jpeg - 4 - 9 - 16
Error: 10695896_page_002.jpeg - 6 - 9 - 18
Error: 10695896_page_002.jpeg - 9 - 6 - 21
Error: 10695896_page_026.jpeg - 2 - 8 - 302
Error: 10695896_page_026.jpeg - 5 - 9 - 305
Error: 10695896_page_026.jpeg - 10 - 5 - 310
Error: 10695896_page_026.jpeg - 11 - 5 - 311
Error: 10695896_page_014.jpeg - 1 - 6 - 157
Error: 10695896_page_014.jpeg - 2 - 8 - 158
Error: 10695896_page_024.jpeg - 1 - 6 - 

# Detect word